In [4]:
import os
from dotenv import load_dotenv

# Load variables from .env file
load_dotenv()

# Get the API key
user_key = os.getenv("RIOT_API_KEY")
user_puuid = os.getenv("RIOT_PUUID")

In [ ]:
import requests 
import pandas as pd
import time

# Raivenomace20 information
puuid = user_puuid 
mass_region = "americas"
# WILL NOT WORK AFTER TODAY. API CHANGES EVERYDAY UNLESS RIOT DEVELOPER WHICH I HAVE NOT BEEN APPROVED OF. SAMPLE STILL REMAINS
api_key = user_key

# The function to get a list of all the match IDs (2nd example above) given a players puuid and mass region
def get_match_ids(puuid, mass_region, api_key):
    api_url_match = (
        "https://" +
        mass_region +
        ".api.riotgames.com/tft/match/v1/matches/by-puuid/" +
        puuid + 
        "/ids?start=0&count=50" + 
        "&api_key=" + 
        api_key
    )
    
    print(api_url_match)
    
    resp = requests.get(api_url_match)
    match_ids = resp.json()
    return match_ids

# From a given match ID and mass region, get the data about the game
def get_match_data(match_id, mass_region, api_key):
    api_url_matchdata = (
        "https://" + 
        mass_region + 
        ".api.riotgames.com/tft/match/v1/matches/" +
        match_id + 
        "?api_key=" + 
        api_key
    )
    
    # we need to add this "while" statement so that we continuously loop until it's successful
    while True:
        resp = requests.get(api_url_matchdata)
        
        # whenever we see a 429, we sleep for 10 seconds and then restart from the top of the "while" loop
        if resp.status_code == 429:
            print("Rate Limit hit, sleeping for 10 seconds")
            time.sleep(10)
            continue
            
        # if resp.status_code isn't 429, then we carry on to the end of  the function and return the data
        match_data = resp.json()
        return match_data


def gather_all_data(puuid, match_ids, mass_region, api_key):
    # We initialise an empty dictionary to store data for each game
    data = {
        'set_number': [],
        'placement': [],
        'level': [],
        'traits': [],
        'units': [],
        'items': []
    }
    
    for match_id in match_ids:
        # run the two functions to get the player data from the match ID
        match_data = get_match_data(match_id, mass_region, api_key)
        set_number = match_data['info']['tft_set_number']
        data['set_number'].append(set_number)
        # assign the variables we're interested in
        index = None
        for pos, id in enumerate(match_data['metadata']['participants']):
            if puuid == id:
                index = pos
                break
        # print(puuid, match_data['metadata']['participants'][index])
        placement = match_data['info']['participants'][index]['placement']
        level = match_data['info']['participants'][index]['level']
        traits = [trait['name'] for trait in match_data['info']['participants'][index]['traits']]
        units = [unit['character_id'] for unit in match_data['info']['participants'][index]['units']]
        items = [item['itemNames'] for item in match_data['info']['participants'][index]['units']]
        # add them to our dataset
        data['placement'].append(placement)
        data['level'].append(level)
        data['traits'].append(traits)
        data['units'].append(units)
        data['items'].append(items)
    
    df = pd.DataFrame(data)
 
    return df

match_ids = get_match_ids(puuid, mass_region, api_key)
df = gather_all_data(puuid, match_ids, mass_region, api_key)
df = df[df['set_number'] == 13].copy()

print(df.head())

https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/oeUTUlTvQIJUxqO955b0viyfcv_-2zvQgTSdhgFJg1nTNJpPSMgtu65dKhC780TONNCU91gxAdNjdQ/ids?start=0&count=50&api_key=RGAPI-c8545638-7a1b-47c7-8080-c7ead206c4a7
    set_number  placement  level  \
14          13          3      9   
15          13          7      9   
16          13          3      8   
17          13          2      9   
19          13          2      9   

                                               traits  \
14  [TFT13_Ambassador, TFT13_Bruiser, TFT13_Cabal,...   
15  [TFT13_Academy, TFT13_Ambassador, TFT13_Bruise...   
16  [TFT13_Ambassador, TFT13_Bruiser, TFT13_Cabal,...   
17  [TFT13_Academy, TFT13_FormSwapper, TFT13_Invok...   
19  [TFT13_Ambusher, TFT13_Bruiser, TFT13_Crime, T...   

                                                units  \
14  [TFT13_Zyra, TFT13_Urgot, TFT13_NunuWillump, t...   
15  [TFT13_Tristana, tft13_swain, TFT13_Nami, TFT1...   
16  [TFT13_Vladimir, TFT13_Tristana, tft13_swain, ...

In [6]:
df.to_excel("tft_analysis.xlsx", index=False)

In [ ]:
""" import requests

url = "https://raw.communitydragon.org/latest/cdragon/tft/en_us.json"
response = requests.get(url)

with open("en_us.json", "w", encoding="utf-8") as f:
    f.write(response.text)

print("Downloaded en_us.json") """

Downloaded en_us.json


In [61]:
import json

# Load the JSON
with open("en_us.json", "r", encoding="utf-8") as f:
    cd_data = json.load(f)

# Extract all valid trait definitions
# Based on:
# 1. apiName starts with "TFT13_"
# 2. associatedTraits is not present (so it's not just a reference)
# 3. name and desc exist (i.e. meaningful definition)
# So, it can include outputs that are not only just traits (i.e. augments etc)
def extract_traits(json_obj):
    if isinstance(json_obj, dict):
        if (
            json_obj.get("apiName", "").startswith("TFT13_")
            and "desc" in json_obj
            and not json_obj.get("associatedTraits")
        ):
            yield json_obj
        for v in json_obj.values():
            yield from extract_traits(v)
    elif isinstance(json_obj, list):
        for item in json_obj:
            yield from extract_traits(item)

# Use dictionary to de-duplicate by apiName
trait_map = {}
for trait in extract_traits(cd_data):
    trait_map[trait["apiName"]] = trait["name"]

# Print final deduplicated result
print("Unique TFT13 Traits:\n")
for api, name in sorted(trait_map.items()):
    print(f"{api}: {name}")


Unique TFT13 Traits:

TFT13_Academy: Academy
TFT13_Ambassador: Emissary
TFT13_Ambusher: Ambusher
TFT13_AnotherAnomaly_Item: Portable Anomaly
TFT13_Augment_AGoldenFind: A Golden Find
TFT13_Augment_AnotherAnomaly: Another Anomaly
TFT13_Augment_Grant6Cost_MissMage: A Change Of Fate
TFT13_Augment_Grant6Cost_Viktor: Glorious Evolution
TFT13_Augment_Grant6Cost_Warwick: What You Really Are
TFT13_Augment_GuidedAnomaly: Guided Anomaly
TFT13_BloodHunter: Blood Hunter
TFT13_Bruiser: Bruiser
TFT13_Cabal: Black Rose
TFT13_Challenger: Quickstriker
TFT13_Crime: Chem-Baron
TFT13_Crime_Gold: Gold
TFT13_Experiment: Experiment
TFT13_Family: Family
TFT13_FormSwapper: Form Swapper
TFT13_GoopBuff_Miniaturize_Item: Miniature Champion
TFT13_GoopBuff_VoraciousAppetite_Consumable: A Chromatic Snack
TFT13_Hextech: Automata
TFT13_HighRoller: High Roller
TFT13_Hoverboard: Firelight
TFT13_Infused: Dominator
TFT13_Invoker: Visionary
TFT13_Item_AcademyEmblemItem: Academy Emblem
TFT13_Item_AmbusherEmblemItem: Ambusher

In [ ]:
# Manually fix naming as Riot has hidden names for certain traits
manual_trait_map = {
    "TFT13_Warband": "TFT13_Conqueror",
    "TFT13_Pugilist": "Pit Fighter",
    "TFT13_Cabal": "TFT13_Black Rose",
    "TFT13_Ambassador": "TFT13_Emissary",
    "TFT13_BloodHunter": "TFT13_Blood Hunter",
    "TFT13_Challenger": "TFT13_Quickstriker",
    "TFT13_Crime": "TFT13_Chem-Baron",
    "TFT13_FormSwapper": "TFT13_Form Swapper",
    "TFT13_Infused": "TFT13_Dominator",
    "TFT13_Invoker": "TFT13_Visionary",
    "TFT13_JunkerKing": "TFT13_Junker King",
    "TFT13_Martialist": "TFT13_Artillerist",
    "TFT13_Squad": "TFT13_Enforcer",
    "TFT13_Titan": "TFT13_Sentinel"

}

def rename_traits(trait_list):
    renamed = []
    for trait in trait_list:
        key = trait
        if key in manual_trait_map:
            renamed.append(manual_trait_map[key])
        else:
            renamed.append(trait)  # fallback to original if not in map
    return renamed

df['traits_original'] = df['traits'].copy(deep=True)
df['traits_renamed'] = df['traits_original'].apply(rename_traits)

print(df[['traits_original', 'traits_renamed']].head(5))

                                      traits_original  \
14  [TFT13_Ambassador, TFT13_Bruiser, TFT13_Cabal,...   
15  [TFT13_Academy, TFT13_Ambassador, TFT13_Bruise...   
16  [TFT13_Ambassador, TFT13_Bruiser, TFT13_Cabal,...   
17  [TFT13_Academy, TFT13_FormSwapper, TFT13_Invok...   
19  [TFT13_Ambusher, TFT13_Bruiser, TFT13_Crime, T...   

                                       traits_renamed  
14  [TFT13_Emissary, TFT13_Bruiser, TFT13_Black Ro...  
15  [TFT13_Academy, TFT13_Emissary, TFT13_Bruiser,...  
16  [TFT13_Emissary, TFT13_Bruiser, TFT13_Black Ro...  
17  [TFT13_Academy, TFT13_Form Swapper, TFT13_Visi...  
19  [TFT13_Ambusher, TFT13_Bruiser, TFT13_Chem-Bar...  


In [ ]:
# Apply renaming to df
df['traits'] = df['traits'].apply(rename_traits)

In [64]:
# TRAITS
import pandas as pd
from collections import Counter

# Step 1: Count all trait appearances
trait_counts = Counter([trait for traits in df['traits'] for trait in traits])

# Step 2: Filter out traits with <=10 total appearances
valid_traits = {trait for trait, count in trait_counts.items() if count > 10}

# Step 3: Create a new column with filtered traits
df['filtered_traits'] = df['traits'].apply(lambda traits: [t for t in traits if t in valid_traits])

# Step 4: Subset into top 4 and bottom 4 using the filtered traits
df_top4 = df[df['placement'] <= 4].copy()
df_bot4 = df[df['placement'] > 4].copy()


In [71]:
# Win/Lose Percentages
from collections import defaultdict

# Initialize counters
trait_top_count = defaultdict(int)
trait_bot_count = defaultdict(int)
trait_total_count = defaultdict(int)

# Count trait stats across filtered data
for _, row in df.iterrows():
    is_top4 = row['placement'] <= 4
    is_bot4 = not is_top4

    for trait in row['filtered_traits']:
        trait_total_count[trait] += 1
        if is_top4:
            trait_top_count[trait] += 1
        else:
            trait_bot_count[trait] += 1

# Create summary DataFrame
trait_data = []
for trait in trait_total_count:
    top_rate = trait_top_count[trait] / trait_total_count[trait]
    bot_rate = trait_bot_count[trait] / trait_total_count[trait]
    trait_data.append({
        'Trait': trait,
        'Top 4 Rate': top_rate,
        'Bottom 4 Rate': bot_rate,
        'Games Played': trait_total_count[trait]
    })

trait_df = pd.DataFrame(trait_data)

top_4_df = trait_df.sort_values(by='Top 4 Rate', ascending=False).head(10)
print("\nTop 10 Traits (Top 4 Placement):")
print(top_4_df)

bot_4_df = trait_df.sort_values(by='Bottom 4 Rate', ascending=False).head(10)
print("\nTop 10 Traits (Bottom 4 Placement):")
print(bot_4_df)


Top 10 Traits (Top 4 Placement):
                 Trait  Top 4 Rate  Bottom 4 Rate  Games Played
6    TFT13_Junker King    0.714286       0.285714            14
13       TFT13_Academy    0.666667       0.333333            12
3   TFT13_Form Swapper    0.562500       0.437500            16
4      TFT13_Dominator    0.562500       0.437500            16
11      TFT13_Sentinel    0.518519       0.481481            27
16         TFT13_Rebel    0.500000       0.500000            24
15     TFT13_Conqueror    0.500000       0.500000            32
5      TFT13_Visionary    0.500000       0.500000            18
1        TFT13_Bruiser    0.500000       0.500000            12
10      TFT13_Enforcer    0.500000       0.500000            12

Top 10 Traits (Bottom 4 Placement):
                 Trait  Top 4 Rate  Bottom 4 Rate  Games Played
14  TFT13_Quickstriker    0.400000       0.600000            20
12       TFT13_Watcher    0.454545       0.545455            22
8          TFT13_Scrap    0.45454

In [ ]:
def run_apriori(df_subset, min_support=0.4, max_confidence=0.99, df_all=None):
    from mlxtend.preprocessing import TransactionEncoder
    from mlxtend.frequent_patterns import apriori, association_rules

    transactions = df_subset['filtered_traits'].tolist()

    # Encode transactions
    te = TransactionEncoder()
    te_array = te.fit_transform(transactions)
    trait_matrix = pd.DataFrame(te_array, columns=te.columns_)

    # Apriori
    frequent_itemsets = apriori(trait_matrix, min_support=min_support, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

    # Filter bad rules
    rules = rules[
        (rules['confidence'] < max_confidence) &
        (rules['antecedents'] != rules['consequents']) &
        rules.apply(lambda row: row['antecedents'].isdisjoint(row['consequents']), axis=1)
    ]

    # Add Top 4 / Bottom 4 rate for rule combinations
    if df_all is not None:
        top4_set = df_all[df_all['placement'] <= 4]['filtered_traits']
        bot4_set = df_all[df_all['placement'] > 4]['filtered_traits']

        def calc_rate(row_set, traits):
            return sum(set(traits).issubset(set(r)) for r in row_set) / len(row_set)

        top4_rates = []
        bot4_rates = []

        for _, rule in rules.iterrows():
            traits = sorted(rule['antecedents'] | rule['consequents'])
            top4_rates.append(calc_rate(top4_set, traits))
            bot4_rates.append(calc_rate(bot4_set, traits))

        rules['Top 4 Rate'] = top4_rates
        rules['Bottom 4 Rate'] = bot4_rates

    return rules[['antecedents', 'consequents', 'support', 'confidence', 'lift', 'Top 4 Rate', 'Bottom 4 Rate']].sort_values(by='Top 4 Rate', ascending=False)

# Use the full dataset as `df_all` for context
top4_rules_df = run_apriori(df_top4, df_all=df)
print(top4_rules_df.head())
bot4_rules_df = run_apriori(df_bot4, df_all=df)
print(bot4_rules_df.head())

                           antecedents                     consequents  \
32                   (TFT13_Conqueror)                   (TFT13_Rebel)   
297                  (TFT13_Conqueror)   (TFT13_Rebel, TFT13_Sentinel)   
295  (TFT13_Conqueror, TFT13_Sentinel)                   (TFT13_Rebel)   
299                   (TFT13_Sentinel)  (TFT13_Conqueror, TFT13_Rebel)   
71                    (TFT13_Sentinel)                   (TFT13_Rebel)   

      support  confidence      lift  Top 4 Rate  Bottom 4 Rate  
32   0.705882    0.750000  1.062500    0.705882       0.555556  
297  0.705882    0.750000  1.062500    0.705882       0.555556  
295  0.705882    0.923077  1.307692    0.705882       0.555556  
299  0.705882    0.857143  1.214286    0.705882       0.555556  
71   0.705882    0.857143  1.214286    0.705882       0.611111  
                       antecedents        consequents   support  confidence  \
18               (TFT13_Conqueror)   (TFT13_Sentinel)  0.666667    0.750000   
19     

In [7]:
# ITEMS
from collections import defaultdict

# Initialize a dictionary to map units to items and count occurrences
unit_item_count = defaultdict(lambda: defaultdict(int))

# Populate the dictionary with units and their corresponding items
for index, row in df.iterrows():
    for unit, items in zip(row['units'], row['items']):
        for item in items:
            unit_item_count[unit][item] += 1

# Sort by descending
sorted_item_count = {unit: item_counts for unit, item_counts in unit_item_count.items() if unit in dict(sorted_unit_count)}
            
# Print the item counts for each unit
for unit, items in sorted_item_count.items():
    sorted_items = sorted(items.items(), key=lambda x: x[1], reverse=True)
    print(f"\nMost common items for unit '{unit}':")
    for item, count in sorted_items:
        print(f"{item}: {count}")



Most common items for unit 'TFT14_Mordekaiser':
TFT14_Item_BallistekEmblemItem: 1
TFT_Item_LocketOfTheIronSolari: 1
TFT_Item_ThiefsGloves: 1
TFT_Item_JeweledGauntlet: 1
TFT_Item_RunaansHurricane: 1
TFT_Item_FrozenHeart: 1

Most common items for unit 'TFT14_MissFortune':
TFT_Item_InfinityEdge: 3
TFT_Item_LastWhisper: 2
TFT_Item_Deathblade: 2
TFT_Item_MadredsBloodrazor: 1
TFT_Item_UnstableConcoction: 1
TFT_Item_PowerGauntlet: 1
TFT_Item_TearOfTheGoddess: 1
TFT_Item_HextechGunblade: 1
TFT_Item_ZekesHerald: 1
TFT_Item_RecurveBow: 1
TFT4_Item_OrnnMuramana: 1

Most common items for unit 'TFT14_Darius':
TFT_Item_IonicSpark: 1

Most common items for unit 'TFT13_Garen':
TFT_Item_Redemption: 22
TFT_Item_WarmogsArmor: 22
TFT_Item_RedBuff: 22
TFT_Item_GargoyleStoneplate: 10
TFT_Item_DragonsClaw: 6
TFT_Item_RadiantVirtue: 2
TFT_Item_NightHarvester: 2
TFT4_Item_OrnnAnimaVisage: 1
TFT_Item_AegisOfTheLegion: 1
TFT5_Item_RedemptionRadiant: 1
TFT_Item_BrambleVest: 1
TFT_Item_Bloodthirster: 1
TFT_Item_U